[View in Colaboratory](https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/nlp/rnn-add-example.ipynb)

In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [23]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.11.0-rc2


In [24]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [25]:
# a small sanity check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


In [26]:
from tensorflow import keras
print(keras.__version__)

2.1.6-tf


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [29]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [30]:
questions[0]

'6+110  '

In [31]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [32]:
len(x[0])

7

In [33]:
len(questions[0])

7

In [34]:
questions[0]

'6+110  '

In [35]:
x[0]

array([[False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [36]:
y[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [37]:
expected[0]

'116 '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [39]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


### Step 3: Create Model

In [40]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [41]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 4, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.predict(np.array([x_val[0]]))

array([[[0.03990376, 0.08339092, 0.10174882, 0.0690695 , 0.11191454,
         0.06986454, 0.0719152 , 0.12844141, 0.03216231, 0.08586498,
         0.08835057, 0.11737339],
        [0.06463561, 0.08647179, 0.10332599, 0.05021508, 0.07692068,
         0.08913328, 0.05807737, 0.15744036, 0.04143989, 0.1009753 ,
         0.06894982, 0.10241491],
        [0.06044635, 0.05534412, 0.08035141, 0.04345266, 0.09017674,
         0.11381816, 0.07136583, 0.21808359, 0.04540214, 0.08416803,
         0.0657936 , 0.07159729],
        [0.05927669, 0.05155819, 0.07752077, 0.0526554 , 0.11733301,
         0.10098131, 0.08565692, 0.20166811, 0.05360102, 0.08825838,
         0.04696324, 0.06452689]]], dtype=float32)

In [43]:
model.predict_classes(np.array([x_val[0]]))

array([[7, 7, 7, 7]])

### Step 4: Train

In [44]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 5s 121us/step - loss: 1.6843 - acc: 0.3875 - val_loss: 1.5620 - val_acc: 0.4284
Q 343+18  T 361  ☒ 326 
Q 53+532  T 585  ☒ 666 
Q 957+28  T 985  ☒ 900 
Q 72+549  T 621  ☒ 762 
Q 19+77   T 96   ☒ 118 
Q 94+519  T 613  ☒ 604 
Q 788+200 T 988  ☒ 1777
Q 54+795  T 849  ☒ 771 
Q 826+212 T 1038 ☒ 120 
Q 607+433 T 1040 ☒ 717 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 5s 111us/step - loss: 1.4533 - acc: 0.4645 - val_loss: 1.3659 - val_acc: 0.4964
Q 951+9   T 960  ☒ 989 
Q 45+225  T 270  ☒ 389 
Q 134+576 T 710  ☒ 177 
Q 10+141  T 151  ☒ 211 
Q 116+46  T 162  ☒ 253 
Q 677+80  T 757  ☒ 779 
Q 56+64   T 120  ☒ 111 
Q 981+956 T 1937 ☒ 1895
Q 82+51   T 133  ☒ 111 
Q 28+714  T 742  ☒ 773 

-------------------

In [45]:
x_val[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [46]:
model.predict(np.array([x_val[0]]))

array([[[2.79526109e-08, 8.51895265e-09, 6.70795816e-06, 1.77009236e-02,
         9.68226492e-01, 1.40420068e-02, 2.21580922e-05, 1.37661075e-07,
         5.23054577e-09, 2.04132999e-09, 1.12506813e-08, 1.60851346e-06],
        [4.10865342e-10, 4.51209692e-09, 1.01068029e-02, 3.87964636e-01,
         5.59736788e-01, 4.18978445e-02, 2.39486151e-04, 1.12083796e-06,
         6.99106195e-08, 7.76496094e-08, 2.84406156e-07, 5.29415593e-05],
        [6.73183315e-11, 7.59089413e-15, 2.87622599e-11, 5.65917162e-06,
         1.37217447e-01, 8.62411082e-01, 3.65810702e-04, 8.57151239e-10,
         3.83236245e-15, 8.39013679e-18, 1.33594549e-18, 2.33289206e-16],
        [9.99999404e-01, 5.21824446e-13, 3.79854134e-11, 5.24088728e-09,
         5.95357790e-07, 2.15290505e-08, 1.03019586e-10, 7.87527595e-14,
         2.56646875e-15, 2.87176763e-14, 1.61396422e-13, 5.31796883e-13]]],
      dtype=float32)

In [47]:
model.predict_classes(np.array([x_val[0]]))

array([[4, 4, 5, 0]])